# 模拟鼠标点击
根据坐标模拟鼠标点击，实现办公自动化。
### 目标：
- 用浏览器自动打开一个网址
- 点击鼠标定位
- 模拟鼠标点击、模拟键盘操作
- 判断运行状态
- 获取运行结果
### 需要用到的库
- `pyautogui`：模拟鼠标点击，模拟键盘输入，获取屏幕截图
- `cv2`：对比屏幕固定位置截图图片变化情况，判断运行状态
- `shutil`：移动文件

In [21]:
import webbrowser
import os
from datetime import datetime, timedelta
import pyautogui
import cv2
import time
import shutil

## 打开浏览器并打开网页
### 用默认浏览器打开

In [13]:
url = "https://www.morningstar.cn/quickrank/default.aspx"

# 用默认浏览器打开
os.system(f'start {url}')

0

### 用指定浏览器打开

In [16]:
# 方法一
# 指定浏览器打开，打开失败，原因不知道
# os.system("start chrome" + url) #os.system("start msegde" + url)

# 方法二
# 指定浏览器路径
msegde_path = r'C:\Program Files (x86)\Microsoft\Edge\Application\msedge.exe'

# 使用指定浏览器打开网页
# 注册并指定浏览器
webbrowser.register('msedge', None, webbrowser.BackgroundBrowser(msegde_path))
webbrowser.get('msedge').open(url)

True

## 日期处理
输入一个文本`2023-06-23`日期，转换为日期类型，并返回一个间隔`n`的日期的字符串

In [20]:
def str_date_delta(date_str, delta_days=-1):
    """
    输入一个格式为 2023-06-30 的日期字符串和时间间隔的天数，返回间隔天数的相同格式的字符串日期
    """
    date_object = datetime.strptime(date_str, "%Y-%m-%d")
    new_date = date_object + timedelta(days=delta_days)
    return new_date.strftime("%Y-%m-%d")

## 模拟鼠标、键盘输入

In [3]:
def click_screen(position=(0, 0)):
    pyautogui.click(x=position[0], y=position[1])
    # 每次点击之后休眠0.4秒，避免因为点击过快导致没有加载好
    time.sleep(0.4)

In [12]:
def press_backspace(n=14):
    """
    按压删除键n次
    """
    for _ in range(n):
        pyautogui.press('backspace')
        # 每按一次backspace要休眠0.5秒，否则按的太快是失效
        time.sleep(0.5)

## 监控运行状态

In [8]:
def get_yunxing_status(yunxing_status_path=r"F:\GitHub\notes\06_OfficeAutomation\yunxing_status.jpg"):
    """
    根据坐标截图，并将截图保存到本地，目的是监控页面某个区域的状态
    """
    # 截图，左上角坐标，宽度，高度
    region = (608, 198, 96, 30)
    screenshot = pyautogui.screenshot(region=region)
    screenshot.save(yunxing_status_path)

In [9]:
def img_compare(img1_path, img2_path):
    """
    输入两张照片，计算巴氏距离，计算照片相关度
    """
    img1 = cv2.imread(img1_path)
    img2 = cv2.imread(img2_path)

    gray1 = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY)

    hist1 = cv2.calcHist([gray1], [0], None, [256], [0, 256])
    hist2 = cv2.calcHist([gray2], [0], None, [256], [0, 256])

    # 巴氏距离值越小，相关度越高，最大值为1，最小值为0haod
    similarity = 1 - cv2.compareHist(hist1, hist2, cv2.HISTCMP_BHATTACHARYYA)
    return similarity

In [11]:
def is_action_done(standard_yunxing, yunxing_status_path=r"F:\GitHub\notes\06_OfficeAutomation\yunxing_status.jpg"):
    """
    输入：目标状态的截图路径，实时状态截图的保存路径。
    说明：先要根据坐标截一张目标状态的图片并保存
    """
    get_yunxing_status(yunxing_status_path)
    similarity = img_compare(yunxing_status_path, standard_yunxing)

    # 删除实时运行状态的照片，因为没隔几秒会重新保存一个文件
    os.remove(yunxing_status_path)

    if similarity > 0.8:
        return True
    else:
        return False

In [19]:
def check_done(n=300, standard_yunxing=r"F:\GitHub\notes\06_OfficeAutomation\standard_yunxing.jpg"):
    for i in range(n):
        if is_action_done(standard_yunxing):
            break
        else:
            # 每两秒
            time.sleep(2)

## 根据关键词找文件名

In [13]:
def find_filse_with_keyword(folder_path, keyword):
    """
    输入文件夹路径和关键词
    返回最后一个包含关键词的文件路径
    如果需要返回所有包含关键词的文件路径，就把result改为一个list
    """
    result = ""
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if keyword in file:
                result = os.path.join(root, files)
    return result

## 获取当前鼠标坐标

In [17]:
# 在jupyter中多个屏幕时，将鼠标悬停后用快捷键运营单元格，可以快速获取坐标位置
current_mouse_position = pyautogui.position()
current_mouse_position

Point(x=2436, y=664)

## 移动文件

In [22]:
def move_file(src_path, dst_path):
    try:
        shutil.move(src_path, dst_path)
        print('文件移动成功')
    except FileNotFoundError:
        print("找不到源文件")
    except FileExistsError:
        print("目标文件已存在")

## 执行运行

In [24]:
def query_data(positon, start_date, end_date, sleep_secs=3):
    click_screen(positon)
    click_screen(627, 530)  # 点击编辑框内
    click_screen(656, 215)  # 点击运行

    click_screen(1200, 380)  # 点击第一个输入框的尾部
    press_backspace(12)  # 按压backspace12次
    pyautogui.typewrite(start_date)

    click_screen(1200, 431)  # 点击第二个输入框的尾部
    press_backspace(12)  # 按压backspace12次
    pyautogui.typewrite(end_date)
    click_screen(1270, 538)  # 确定运行

    pyautogui.move(627, 530)  # 移动到编辑框中
    time.sleep(sleep_secs)